In [4]:
import transformers

from transformers import AutoTokenizer
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationChain
from langchain import PromptTemplate, LLMChain
from IPython.display import display, Markdown

In [6]:
model_name = "microsoft/phi-1_5"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [11]:

import torch
from transformers import AutoModelForCausalLM

In [12]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_8bit=False,
    device_map="auto",
    trust_remote_code=True,
)
model = base_model

In [13]:
from datasets import load_dataset

data = load_dataset("truthful_qa", "generation")
data = data["validation"].filter(lambda item: item["category"] == "Misconceptions")
data

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

Filter:   0%|          | 0/817 [00:00<?, ? examples/s]

Dataset({
    features: ['type', 'category', 'question', 'best_answer', 'correct_answers', 'incorrect_answers', 'source'],
    num_rows: 100
})

In [15]:
default_template = """
Human: {input}
AI:"""

verbose_template = """
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: {input}
AI:"""

In [16]:
def get_chain(model, template, verbose=False):
    pipeline = transformers.pipeline(
        model=model,
        tokenizer=tokenizer,
        return_full_text=True,
        task="text-generation",
        stop_sequence="\nAI:",
        temperature=0.7,
        max_new_tokens=512,
        repetition_penalty=1.2,
    )
    return LLMChain(
        llm=HuggingFacePipeline(pipeline=pipeline),
        prompt=PromptTemplate.from_template(template),
        verbose=verbose,
    )

def compare_results(base_model, fine_tuned_model, item, template=None, verbose=False):
    if template is None:
        template = default_template
    base_chain = get_chain(base_model, template, verbose)
    fine_tuned_chain = get_chain(fine_tuned_model, template, verbose)
    base_res = base_chain.run(item["question"])
    fine_tuned_res = fine_tuned_chain.run(item["question"])
    display(
        Markdown(
            f"""
### question:
{item['question']}
#### base_model:
{base_res}
#### fine_tuned_model:
{fine_tuned_res}
#### best answer:
{item['best_answer']}
#### source:
{item['source']}
"""
        )
    )

In [17]:
base_chain_verbose = get_chain(base_model, verbose_template)
fine_tuned_verbose = get_chain(model, verbose_template)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_generation.py:145: UserWarning: Stopping on a multiple token sequence is not yet supported on transformers. The first token of the stop sequence will be used as the stop sequence string in the interim.
  warnings.warn(


In [18]:
compare_results(base_model, model, data[46])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


ValueError: ignored